In [1]:
import numpy as np
import pandas as pd
!pip install seaborn
 
# For visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.options.display.max_rows = None
pd.options.display.max_columns = None

/tmp/ipykernel_533/66417288.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


     |████████████████████████████████| 296kB 6.4MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
pip install --upgrade pip

     |████████████████████████████████| 1.8MB 6.9MB/s eta 0:00:01
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
Note: you may need to restart the kernel to use updated packages.


In [1]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
table_name = 'PHARMASALESANALYSIS'
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

ModuleNotFoundError: No module named 'fosforml'